In [26]:
import cv2
import glob
import random
import numpy as np
from sklearn.utils import shuffle

emotions = ["anger", "disgust", "fear", "happy", "sadness",] #Emotion list

fishface = cv2.face.FisherFaceRecognizer_create() # Initialize fisher face classifier
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

data = {}

def get_files(emotion): # Define function to get file list, randomly shuffle it and split 80/20
    files = glob.glob("CK+48/%s/*" %emotion)
    random.shuffle(files)
    training = files[:int(len(files)*0.8)] # get first 80% of the list
    prediction = files[-int(len(files)*0.2):] # get last 20% of the list
    return training, prediction

def make_sets():
    training_data = []
    training_labels = []
    prediction_data = []
    prediction_labels = []
    for emotion in emotions:
        training, prediction = get_files(emotion)
        # Append data to training and prediction list, and generate labels 0-7
        for item in training:
            image = cv2.imread(item) # open image
            image = cv2.resize(image, (220, 220))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
            clahe_image = clahe.apply(gray)
            training_data.append(clahe_image) # append image array to training data list
            training_labels.append(emotions.index(emotion))
            
        for item in prediction: # Repeat above process for prediction set
            image = cv2.imread(item)
            image = cv2.resize(image, (220, 220))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            clahe_image = clahe.apply(gray)
            prediction_data.append(clahe_image) # append image array to training data list
            prediction_labels.append(emotions.index(emotion))
            
    return training_data, training_labels, prediction_data, prediction_labels
    
def run_recognizer():
    training_data, training_labels, prediction_data, prediction_labels = make_sets()
    
    training_data, training_labels = shuffle(training_data, training_labels)
    prediction_data, prediction_labels = shuffle(prediction_data, prediction_labels)
    print("Training fisher face classifier")
    print("Size of training set: ", len(training_labels), "images")
    model = fishface.train(training_data, np.asarray(training_labels))
    
    print("\nPredicting classification set")
    cnt = 0
    correct = 0
    incorrect = 0
    print(prediction_labels)
    predd=[]
    for image in prediction_data:
        pred, conf = fishface.predict(image)
        predd.append(pred)
        if pred == prediction_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            cv2.imwrite("dataset\\difficult\\%s_%s_%s.jpg" %(emotions[prediction_labels[cnt]], emotions[pred], cnt), image)
            
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect),predd,prediction_labels,prediction_data, model)
        
metascore = []
for i in range(0,1):
    correct, pred, prediction_labels, prediction_data, model = run_recognizer()
    #print("got", correct, "percent correct!")
    metascore.append(correct)
    
print ("\n\nMeta-score:", np.mean(metascore))  


Training fisher face classifier
Size of training set:  541 images

Predicting classification set
[1, 3, 1, 0, 1, 3, 3, 1, 3, 1, 2, 2, 2, 2, 1, 1, 0, 0, 1, 0, 0, 1, 3, 3, 0, 1, 0, 4, 3, 3, 0, 4, 1, 1, 1, 0, 4, 3, 0, 0, 3, 3, 3, 1, 4, 1, 4, 0, 4, 4, 3, 3, 4, 3, 3, 1, 0, 4, 0, 3, 0, 1, 3, 2, 3, 1, 2, 3, 3, 4, 1, 3, 1, 3, 1, 0, 3, 0, 3, 1, 3, 4, 1, 1, 3, 1, 1, 2, 3, 0, 1, 1, 3, 3, 0, 3, 3, 3, 0, 1, 2, 2, 0, 4, 3, 1, 2, 3, 3, 2, 2, 1, 1, 4, 2, 3, 0, 4, 0, 3, 1, 0, 1, 4, 2, 3, 3, 0, 3, 1, 0, 4, 0, 2]


Meta-score: 98.50746268656717


In [27]:
from sklearn.metrics import classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

print("Classification Report :\n{}".format(classification_report(pred, prediction_labels)))
print("Confusion Matrix :") 
print(confusion_matrix(pred, prediction_labels))
print("\nAccuracy : {}".format(accuracy_score(pred, prediction_labels)))

Classification Report :
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        27
           1       1.00      0.95      0.97        37
           2       1.00      1.00      1.00        15
           3       0.95      1.00      0.97        39
           4       1.00      1.00      1.00        16

    accuracy                           0.99       134
   macro avg       0.99      0.99      0.99       134
weighted avg       0.99      0.99      0.99       134

Confusion Matrix :
[[27  0  0  0  0]
 [ 0 35  0  2  0]
 [ 0  0 15  0  0]
 [ 0  0  0 39  0]
 [ 0  0  0  0 16]]

Accuracy : 0.9850746268656716


In [38]:
import cv2
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "facial_expression.png"
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

image = cv2.imread('facial_expression.png',cv2.IMREAD_COLOR)
image = cv2.resize(image, (220, 220))
#Converting the from from BGR to RGB
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
clahe_image = clahe.apply(image)

#model = joblib.load("./face_emotion.joblib")
fishface.write('./facial_emotion.cv2')
npar_pred = fishface.predict(clahe_image)
print(npar_pred)

emoji_dict = {"joy":"😃", "fear":"😱", "anger":"😡", "sadness":"😢", "disgust":"🤢"}
emotion = emotions[npar_pred[0]]
if emotion == "happy":
    emotion = "joy"
    
print("\nPredicted Emotion: " + emoji_dict[emotion] + str(emotion))

print("\n\nRecommended Music Playlist:\n")
with open('lyric_classification_log.txt', 'r') as f:
    for line_number, line in enumerate(f, start=1):
        if emotion in line:
            print(line)

facial_expression.png written!
Escape hit, closing...
(0, 319.33734470894217)

Predicted Emotion: 😡anger


Recommended Music Playlist:

song_6.txt - anger - Waste of Time

song_8.txt - anger - Wars

